# JSON validator

In this project our goal is to validate one dictionary structure against a template dictionary.

For example you might have this template:

In [2]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

So, a JSON document such as this would match the template:

In [3]:
john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

But this one would **not** match the template (missing key):

In [4]:
eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

And neither would this one (wrong data type):

In [5]:
michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

Write a function such this:

In [ ]:
def validate(data, template):
    # implement
    # and return True/False
    # in the case of False, return a string describing 
    # the first error encountered
    # in the case of True, string bbcan be empty
    return state, error

In [68]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys
    
    if missing_keys or extra_keys:
        is_ok = False
        missing_msg = (f'missing keys: \n{", ".join({path + "." + str(key) for key in missing_keys})}') if missing_keys else ''
        extras_msg = (f'extra keys: \n{", ".join({path + "." + str(key) for key in extra_keys})}') if extra_keys else ''
        return False, ''.join(('\n', missing_msg , '\n', extras_msg))
    else:
        return True, None
                                        

In [69]:
# test for match_keys
d = {'a':1, 'b':2, 'c':3, 'f':0}
t = {'a':1, 'e':None, 'c':{'d':4}}

is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

False 
missing keys: 
some.path.e
extra keys: 
some.path.f, some.path.b


In [81]:
def match_types(data, template, path):
    for key, value, in template.items():
        if isinstance(value, dict): 
            # check if the values is actually another dict to recurse the cheking operations
            template_type = dict
        else:
            template_type = value
            
        data_value = data.get(key, object()) # default to object becasue any of the template data can be one
        if not isinstance(data_value, template_type):
            err_msg = (f'\nincorrect type: {path + "." + key} \n-> expected {template_type.__name__} \n-> found {type(data_value).__name__}')
            return False, err_msg
    return True, None

In [83]:
# test for match_types
d = {'a':1, 'b':'str', 'c':{}}
t = {'a':int, 'b':int, 'c':int}

is_ok, err_msg = match_types(d, t, 'some.path')
print(is_ok, err_msg)

False 
incorrect type: some.path.b 
-> expected int 
-> found str


In [85]:
def recurse_validate(data, template, path):
    is_ok, err_msg = match_keys(data, template, path)
    if not is_ok:
        return is_ok, err_msg
    
    is_ok, err_msg = match_types(data, template, path)
    if not is_ok:
        return is_ok, err_msg
    
    # Now we need to check if among the template keys there are any dictionaries to iterate the process
    dict_type_keys = {key for key, value in template.items() if isinstance(value, dict)}
    
    for key in dict_type_keys:
        sub_path = path + '.' + str(key)
        sub_template = template[key]
        sub_data = data[key]
        is_ok, err_msg = recurse_validate(sub_data, sub_template, sub_path)
        if not is_ok:
            return is_ok, err_msg
        
    return True, None

In [87]:
is_ok, err_msg = recurse_validate(john, template, 'john')
print(is_ok, err_msg)

True None


In [88]:
is_ok, err_msg = recurse_validate(eric, template, 'eric')
print(is_ok, err_msg)

False 
missing keys: 
eric.bio.birthplace.city



In [89]:
is_ok, err_msg = recurse_validate(michael, template, 'micheal')
print(is_ok, err_msg)

False 
incorrect type: micheal.bio.dob.month 
-> expected int 
-> found str


In [95]:
def validate(data, template):
    is_ok, err_msg = recurse_validate(data, template, 'root')
    return is_ok, err_msg

In [96]:
is_ok, err_msg = validate(eric, template,)
print(is_ok, err_msg)

False 
missing keys: 
root.bio.birthplace.city

